# Python for Webscraping
* SOC 590: Big Data and Population Processes
* 17th October 2016

## Tutorial 1: Webscraping a table to .csv

In [1]:
import os
import csv
import urllib
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
if not os.path.exists('../data/'):
    os.makedirs('../data/')

In [3]:
soup = BeautifulSoup(urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_most_popular_websites'), "lxml")
headers = [header.text for header in soup.findAll('th')]
rows = []
headers
for row in soup.find_all('tr')[1:]:
    rows.append([val.text for val in row.find_all_next('td',limit=6)])
    

with open('../data/wiki_popular_sites.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(row for row in rows if row)

In [4]:
wiki_popular_sites_ = pd.read_csv("../data/wiki_popular_sites.csv")
wiki_popular_sites_.head(10)

,Site,Domain,"Alexa top 100 websites (As of September 4, 2016)[3]","SimilarWeb top 100 websites (As of September 4, 2016)[4]",Type,Principal country
0,Google,google.com,1,2,Internet services and products,U.S.
1,YouTube,youtube.com,2,3,Video sharing,U.S.
2,Facebook,facebook.com,3,1,Social network,U.S.
3,Baidu,baidu.com,4,16,Search engine,China
4,Yahoo!,yahoo.com,5,5,Portal and media,U.S.
5,Amazon,amazon.com,6,10,E-commerce and cloud computing,U.S.
6,Wikipedia,wikipedia.org,7,7,Encyclopedia,U.S.
7,Tencent QQ,qq.com,8,41,Portal,China
8,Google India,google.co.in,9,14,Search engine,India
9,Twitter,twitter.com,10,8,Social network,U.S.


In [5]:
wiki_popular_sites_.columns

Index(['Site', 'Domain',
       'Alexa top 100 websites\n(As of September 4, 2016)[3]',
       'SimilarWeb top 100 websites\n(As of September 4, 2016)[4]', 'Type',
       '\nPrincipal country\n'],
      dtype='object')

In [6]:
grouped_sites = wiki_popular_sites_.groupby(['Type','\nPrincipal country\n']).count()

In [7]:
grouped_sites.iloc[:,0:1]

,,Site
Type,Principal country,
Ad serving,U.S.,1
Advertising,U.S.,1
Blogging,U.S.,2
Blogging and social media,U.S.,1
Booking engine,Netherlands,1
Classified advertising,U.S.,1
Content marketing,U.S.,1
Disney portal,U.S.,1
E-commerce,China,1


In [8]:
wiki_popular_sites_['Type'].unique()

array(['Internet services and products', 'Video sharing', 'Social network',
       'Search engine', 'Portal and media',
       'E-commerce and cloud computing', 'Encyclopedia', 'Portal',
       'Email, web services and software suite', 'Online shopping',
       'Photo sharing and social media', 'Portal and instant messaging',
       'Professional Social network', 'Web directories',
       'social news networking, entertainment',
       'Online auctions and shopping',
       'URL shortening for links on Twitter', 'Retail',
       'Internet security and anti-trojan software', 'E-commerce',
       'Social media', 'Online advertising network',
       'Streaming TV and movies', 'Software and technology',
       'Blogging and social media', 'Newspaper', 'Payment system',
       'Blogging', 'Image sharing', 'Question and answer site',
       'Social Networking', 'Technology and software',
       'Source code hosting service', 'Search Engine', 'Pornography',
       'Film, TV show, and video ga